# Title
[]()

In [2]:

import pandas as pd
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\custom_python")
from silvhua import *
# from datetime import datetime
# sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\src")

# From `data_functions.py`

In [2]:
import boto3
from boto3.dynamodb.conditions import Key
import json
from datetime import datetime, timezone
import sys
import os

def query_dynamodb_table(table_name, value, key='SessionId', region_name="us-west-2"):
    """
    Query a Dynamodb table and print the results.
    Parameters:
        table_name (str): The name of the DynamoDB table.
        value (str): The value of the key.
        key (str): The name of the key.
        region_name (str): The name of the AWS region.

    Returns:
        dict: The results of the query. The 'Items' key contains the table record.       
    From 2023-10-22 notebook.
    """
    dynamodb = boto3.resource('dynamodb', region_name=region_name)
    table = dynamodb.Table(table_name)
    response = table.query(
        KeyConditionExpression=Key(key).eq(value)
    )
    return response

# Payload

In [3]:
event = {
  "body": {
    "type": "InboundMessage",
    "locationId": "QSZ6lvarPNxtmTFl84W8",
    "attachments": [],
    "body": "This is another test inbound message",
    "contactId": "test",
    "contentType": "text/plain",
    "conversationId": "fcanlLgpbQgQhderivVs",
    "dateAdded": "2023-11-10T12:26:02.000Z",
    "direction": "inbound",
    "messageType": "SMS",
    "userId": "80pzXdC3MCqKRpzrteuj"
  }
}
payload = event["body"]

# Iteration 1 

In [5]:
message_events = ['InboundMessage', 'OutboundMessage', 'NoteCreate']
contact_id_key = 'contactId' if payload['type'] in message_events else 'id'
contact_data = query_dynamodb_table(
    'SessionTable', payload[contact_id_key], key='SessionId'
    )

In [6]:
contact_data

{'Items': [],
 'Count': 0,
 'ScannedCount': 0,
 'ResponseMetadata': {'RequestId': 'MRGSD8EQ2SJUUE7JO93SRR6LOJVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Sat, 11 Nov 2023 20:21:42 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '39',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'MRGSD8EQ2SJUUE7JO93SRR6LOJVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '3413411624'},
  'RetryAttempts': 0}}

In [8]:
if contact_data['Items']:
    print(contact_data['Items'])
else:
    print('No contact data found')

No contact data found


# *End of Page*